### Exercise 2
#### 导入

In [68]:
# Imports
import requests  # send request
from bs4 import BeautifulSoup  # parse web pages
import pandas as pd  # csv
import time
import random


#### Headers

In [69]:
# Define a request header (to prevent anti-scraping)
headers = {
    'authority': 'curlconverter.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'max-age=0',
    'if-modified-since': 'Fri, 15 Jul 2022 21:44:42 GMT',
    'if-none-match': 'W/"62d1dfca-3a58"',
    'referer': 'https://curlconverter.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Microsoft Edge";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30',
}

#### 单页爬取

In [70]:
# # YOUR CODE HERE
# # Step 1: Define the URL
# url = "https://arxiv.org/list/cs/recent"
# 
# # Step 2: Open the URL using requests.get() with headers
# page = requests.get(url, headers=headers)
# 
# # Step 3: Parse the HTML content using BeautifulSoup
# soup = BeautifulSoup(page.content, 'lxml')

In [72]:
# title_list = []
# subjects_list = []
# authors_list = []
# abstract_list = []
# website_url_list = []
# i = 1
# for dd in soup.dl.find_all('dd'):
#     # print(dd.prettify())
#     title_list.append(dd.find_all('div', class_='list-title mathjax')[0].text.strip().split('Title: ')[1])
#     subjects_list.append(dd.find_all('div', class_='list-subjects')[0].text.strip().split('Subjects: ')[1])
#     authors_list.append(dd.find_all('div', class_='list-authors')[0].text.strip().split('Authors:\n')[1].split(", \n"))
# 
# for dt in soup.dl.find_all('dt'):
#     website_url_list.append("https://arxiv.org" + dt.find_all('span')[0].find_all('a')[0].get('href'))
# 
# for website_url in website_url_list:
#     print(i)
#     i = i + 1
#     try:
#         time.sleep(random.uniform(3, 5))
#         website_page = requests.get(website_url, headers=headers)
#         website_soup = BeautifulSoup(website_page.content, 'lxml')
#         abstract = website_soup.find('blockquote').text.strip().split('Abstract:')[1]
#         abstract_list.append(abstract)
#     except Exception as e:
#         abstract_list.append('error')
#         print(f"在爬取{website_url}时发生错误：{e}")

#### 多页爬取

In [73]:
title_list = []
subjects_list = []
authors_list = []
abstract_list = []
website_url_list = []

##### 读取网页

In [74]:
for i in range(0, 4):
    url = "https://arxiv.org/list/cs/pastweek?skip=" + str(i * 25) + "&show=25"
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'lxml')
    for dd in soup.dl.find_all('dd'):
        title_list.append(dd.find_all('div', class_='list-title mathjax')[0].text.strip().split('Title: ')[1])
        subjects_list.append(dd.find_all('div', class_='list-subjects')[0].text.strip().split('Subjects: ')[1])
        authors_list.append(
            dd.find_all('div', class_='list-authors')[0].text.strip().split('Authors:\n')[1].split(", \n"))
    
    for dt in soup.dl.find_all('dt'):
        website_url = "https://arxiv.org" + dt.find_all('span')[0].find_all('a')[0].get('href')
        website_url_list.append(website_url)
        
        try:
            time.sleep(random.uniform(1, 3))
            website_page = requests.get(website_url, headers=headers)
            website_soup = BeautifulSoup(website_page.content, 'lxml')
            abstract = website_soup.find('blockquote').text.strip().split('Abstract:')[1]
            abstract_list.append(abstract)
        except Exception as e:
            abstract_list.append('error')
            print(f"在爬取{website_url}时发生错误：{e}")

1:https://arxiv.org/abs/2403.00762
2:https://arxiv.org/abs/2403.00758
3:https://arxiv.org/abs/2403.00752
4:https://arxiv.org/abs/2403.00745
5:https://arxiv.org/abs/2403.00743
6:https://arxiv.org/abs/2403.00742
7:https://arxiv.org/abs/2403.00737
8:https://arxiv.org/abs/2403.00729
9:https://arxiv.org/abs/2403.00725
10:https://arxiv.org/abs/2403.00724
11:https://arxiv.org/abs/2403.00720
12:https://arxiv.org/abs/2403.00717
13:https://arxiv.org/abs/2403.00715
14:https://arxiv.org/abs/2403.00712
15:https://arxiv.org/abs/2403.00704
16:https://arxiv.org/abs/2403.00696
17:https://arxiv.org/abs/2403.00691
18:https://arxiv.org/abs/2403.00690
19:https://arxiv.org/abs/2403.00689
20:https://arxiv.org/abs/2403.00686
21:https://arxiv.org/abs/2403.00685
22:https://arxiv.org/abs/2403.00682
23:https://arxiv.org/abs/2403.00680
24:https://arxiv.org/abs/2403.00675
25:https://arxiv.org/abs/2403.00674
26:https://arxiv.org/abs/2403.00673
27:https://arxiv.org/abs/2403.00669
28:https://arxiv.org/abs/2403.00668
2

##### 保存文件

In [78]:
csv_name = "CSTopic.csv"
csv_dir = "./output"
file_path = f"{csv_dir}/{csv_name}"

CS_df = pd.DataFrame({
    'Title': title_list,
    'Subjects': subjects_list,
    'Authors': authors_list,
    'Abstract': abstract_list
})
# print(CS_df.head())

CS_df.to_csv(file_path, index=False)

print(f"DataFrame saved to {file_path}")

DataFrame saved to ./output/CSTopic.csv
